In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv(r"C:\Imp\SamanyuK\Datasets\winequalityN.csv")
df

In [ ]:
df.info()

In [ ]:
df=df.dropna()
tdf=df.copy()
from sklearn.feature_selection import mutual_info_classif
x=tdf.drop(columns='type')
y=tdf['type']
mi=mutual_info_classif(x,y)
ndf=pd.DataFrame({'columns':x.columns,'mi':mi})
print(ndf)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
t_c=12
cpr=3
r=4
fig,axes=plt.subplots(cpr,r,figsize=(5*cpr,5*r),sharey=True)
axes=axes.flatten()
for i,col in enumerate(x.columns):
    sns.histplot(x=x[col], ax=axes[i],kde=True)

In [ ]:
t_c=12
cpr=3
r=4
fig,axes=plt.subplots(cpr,r,figsize=(5*cpr,5*r),sharey=True)
axes=axes.flatten()
for i,col in enumerate(x.columns):
    sns.boxplot(x=x[col], ax=axes[i],y=tdf['type'],hue=tdf['type'])

In [ ]:
c=df['type'].value_counts().reset_index()
c.columns=['type','count']
plt.figure(figsize=(8,8))
plt.pie(x=c['count'],labels=c['type'],autopct='%.2f%%')
plt.legend(c['type']+' wine')
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
sns.catplot(data=df,x='quality',kind='count',hue='type')
plt.show()

In [ ]:
count_df = df.groupby(['type', 'quality']).size().reset_index(name='count')
total_counts = df['type'].value_counts().reset_index()
total_counts.columns = ['type', 'total_count']
count_df = pd.merge(count_df, total_counts, on='type')
count_df['percentage'] = (count_df['count'] / count_df['total_count']) * 100

plt.figure(figsize=(12, 8))
sns.barplot(data=count_df, x='quality', y='percentage', hue='type')
plt.ylabel('Percentage')
plt.title('Percentage of Each Wine Type in Different Quality Categories')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [ ]:
pred=[]
accuracy=[]
x=df.drop(columns='type')
y=df['type']
x_t,x_te,y_t,y_te=train_test_split(x,y,test_size=0.25,random_state=20)

**Logistic Regression**

In [ ]:
lx_t=x_t.copy()
lx_te=x_te.copy()
ly_t=y_t.copy()
ly_te=y_te.copy()
lreg=LogisticRegression(max_iter=1000)
param = {'penalty': ['l1', 'l2'], 'solver': ['liblinear', 'saga']}
nlreg=RandomizedSearchCV(lreg,param_distributions=param,n_jobs=-1,cv=10,n_iter=4,scoring='accuracy')
nlreg.fit(lx_t,ly_t)
print(nlreg.best_params_)
print(nlreg.best_score_)

In [ ]:
nlreg=nlreg.best_estimator_
pred1=nlreg.predict(lx_te)
pred.append(pred1)
accuracy.append(accuracy_score(y_te,pred1))
print(accuracy_score(y_te,pred1))

**K Nearest Neighbors**

In [ ]:
kx_t=x_t.copy()
kx_te=x_te.copy()
ky_t=y_t.copy()
ky_te=y_te.copy()
knn=KNeighborsClassifier()
param={'n_neighbors':list(np.arange(1,201,1)), 'weights': ['uniform', 'distance'],'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
      'p': [1, 2]}
nknn=RandomizedSearchCV(knn,param_distributions=param,scoring='accuracy',cv=10,n_jobs=-1)
nknn.fit(kx_t,ky_t)
print(nknn.best_params_)
print(nknn.best_score_)

In [ ]:
nknn=nknn.best_estimator_
pred2=nknn.predict(kx_te)
pred.append(pred2)
accuracy.append(accuracy_score(y_te,pred2))
print(accuracy_score(y_te,pred2))

**Decision Trees**

In [ ]:
dx_t=x_t.copy()
dx_te=x_te.copy()
dy_t=y_t.copy()
dy_te=y_te.copy()
dt=DecisionTreeClassifier()
path=dt.cost_complexity_pruning_path(dx_t,dy_t)
alphas=path.ccp_alphas
param={'ccp_alpha':alphas}
ndt=RandomizedSearchCV(dt,param_distributions=param,cv=10,n_jobs=-1,scoring='accuracy')
ndt.fit(dx_t,dy_t)
ba=ndt.best_params_['ccp_alpha']

In [ ]:
dt=DecisionTreeClassifier(ccp_alpha=ba)
param={'criterion':['gini','entropy'],'min_samples_split':list(np.arange(2,401)),'min_samples_leaf':list(np.arange(2,401)),
       'max_features':['sqrt','log2',None]}
ndt=RandomizedSearchCV(dt,param_distributions=param,cv=10,n_jobs=-1,scoring='accuracy')
ndt.fit(dx_t,dy_t)
print(ndt.best_params_)
print(ndt.best_score_)

In [ ]:
ndt=ndt.best_estimator_
pred3=ndt.predict(dx_te)
pred.append(pred3)
accuracy.append(accuracy_score(y_te,pred3))
print(accuracy_score(y_te,pred3))

**Random Forest Classifier**

In [ ]:
rx_t=x_t.copy()
rx_te=x_te.copy()
ry_t=y_t.copy()
ry_te=y_te.copy()
rf=RandomForestClassifier()
param={'criterion':['gini','entropy'],
       'max_features':['sqrt','log2',None],'n_estimators':[200]}
nrf=RandomizedSearchCV(rf,param_distributions=param,cv=10,n_jobs=-1,scoring='accuracy',n_iter=6)
nrf.fit(rx_t,ry_t)
print(nrf.best_params_)
print(nrf.best_score_)

In [ ]:
nrf=nrf.best_estimator_
pred4=nrf.predict(rx_te)
pred.append(pred4)
accuracy.append(accuracy_score(y_te,pred4))
print(accuracy_score(y_te,pred4))

In [ ]:
label=['Logistic Regression','KNN','Decision Trees','Random Forest']
for i,j,k in zip(pred,label,accuracy):
    print(j,'\n')
    print(f'Accuracy: {k}\n\n')
    print(classification_report(y_te,i),'\n\n')

In [ ]:
fig,axs=plt.subplots(2,2,figsize=(12,10))
cm1=confusion_matrix(y_te,pred[0])
cm2=confusion_matrix(y_te,pred[1])
cm3=confusion_matrix(y_te,pred[2])
cm4=confusion_matrix(y_te,pred[3])
sns.heatmap(cm1,annot=True,ax=axs[0,0],cmap='magma',xticklabels=['White Wine','Red Wine'],yticklabels=['White Wine','Red Wine'])
axs[0, 0].set_title('Confusion Matrix for Logistic Regression')
sns.heatmap(cm2,annot=True,ax=axs[0,1],cmap='magma',xticklabels=['White Wine','Red Wine'],yticklabels=['White Wine','Red Wine'])
axs[0, 1].set_title('Confusion Matrix for K Nearest Neighbors')
sns.heatmap(cm3,annot=True,ax=axs[1,0],cmap='magma',xticklabels=['White Wine','Red Wine'],yticklabels=['White Wine','Red Wine'])
axs[1, 0].set_title('Confusion Matrix for Decision Tree Classifier')
sns.heatmap(cm4,annot=True,ax=axs[1,1],cmap='magma',xticklabels=['White Wine','Red Wine'],yticklabels=['White Wine','Red Wine'])
axs[1, 1].set_title('Confusion Matrix for Random Forest Classifier')
plt.tight_layout()
plt.show()